In [1]:
from service import config
config.initialize()

from service.MatchSys import MatchSys
ms = MatchSys(
        name='teat_sys',
        ltp_model_path=r'D:\Code\MyLongTimeProject\A\QQ-Bot-And-Tool\data\LtpModel',
        database_uri='sqlite:///data/db.sqlite3',
        text_vec_model_path=r'D:\Code\MyLongTimeProject\A\QQ-Bot-And-Tool\data\model.pkl',
        vector_similarity_rate=0.7,
        most_similar_number=10,
        vector_match_times=5,

    )

1714939965038796800
Loading weights from local directory
# jionlp - 微信公众号: JioNLP  Github: `https://github.com/dongrixinyu/JioNLP`.
# jiojio - `http://www.jionlp.com/jionlp_online/cws_pos` is available for online trial.
Parrot started


In [ ]:
import pandas as pd
map = {}
# data = pd.read_excel(r'C:\Users\Administrator\Documents\GitHub\QQ-Bot-And-Tool\data\FixedReply\傲娇系二次元bot词库5千词V1.2.xlsx',header=None, sheet_name=0)
data = pd.read_excel(r"D:\temp\Downloads\傲娇系二次元bot词库5千词V1.2.xlsx",header=None, sheet_name=0)
for index,row in data.iterrows():
    if row[0] in map:
        map[row[0]].append(row[1])
    else:
        map[row[0]] = [row[1]]
from service.MatchSys.trainer import QATrainer
trainer = QATrainer(ms)


In [2]:
trainer.train(map)

NameError: name 'trainer' is not defined

In [3]:
ms.get_response('我有蛋糕你要吗')

[]


IndexError: list index out of range

In [ ]:
ms.docvector_tool.inferred2string('我 有 蛋糕 你 要 吗'.split(' '))

In [ ]:
inferred_vector = ms.docvector_tool.model.infer_vector(doc_words='我 有 蛋糕 你 要 吗'.split(' ')) 
sims = ms.docvector_tool.model.dv.most_similar([inferred_vector],topn=10)
for sim in sims:
    print(sim)

In [ ]:
input_statement = None
response = None
message_adapter = None
message = '我有蛋糕你要吗'
for messageadapter in ms.message_adapters:
    if messageadapter.check(message):
        message_adapter = messageadapter
input_statement = message_adapter.process(message)

input_statement_list = []
for input_statement_id in ms.docvector_tool.inferred2string(input_statement.search_text.split(' ')):
    input_statement_list.append(ms.storage.get_statement_by_id(int(input_statement_id)))
